In [ ]:
# Model Training:
# 1) Load all data from preprocessing (training/test splits, etc)
# 2) Begin Training Models
    #  a) Decision Tree
    #  b) Naive Bayes
    #  c) Logistic Regression
    #  d) SVM
# 3) Testing Models
# 4) New Iterations

In [1]:
# Imports
from Models import ModelUtil
import importlib
from Data import Preprocessing
from sklearn.model_selection import train_test_split

In [2]:
# 1) Load all data from preprocessing 
importlib.reload(Preprocessing)
importlib.reload(ModelUtil)
infieldDataFrame, outfieldDataFrame = Preprocessing.dataFiltering()

c:\Users\Trent\Desktop\Senior Design\shifting_model\Data\DataUtil.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["PitcherThrows"] = df["PitcherThrows"].map({"Left":1, "Right":2, "Both":3})


In [3]:
# 2) Training Models
Y = infieldDataFrame["FieldSlice"]
X = infieldDataFrame[["PitcherThrows", "BatterSide", "TaggedPitchType", "ZoneSpeed", "PlateLocHeight", "PlateLocSide"]]
xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size=0.25, random_state=11)

In [4]:
# a) Decision Tree
# Need to test these hyperparameters for best case
result = ModelUtil.runDT(xTrain, yTrain, xTest, yTest, 25, 10, 55)
print(result)

training decision tree model...
done!
getting statistics...
11
55
[[1.         0.         0.         0.         0.        ]
 [0.0952381  0.66666667 0.19047619 0.         0.04761905]
 [0.125      0.         0.         0.875      0.        ]
 [0.16666667 0.         0.75       0.08333333 0.        ]
 [0.         0.07407407 0.14814815 0.48148148 0.2962963 ]
 [0.         1.         0.         0.         0.        ]
 [0.125      0.625      0.25       0.         0.        ]
 [0.08       0.32       0.36       0.2        0.04      ]
 [0.         0.07407407 0.14814815 0.48148148 0.2962963 ]
 [0.0952381  0.66666667 0.19047619 0.         0.04761905]
 [0.         0.         0.         0.         1.        ]
 [0.         0.         0.28571429 0.71428571 0.        ]
 [0.08       0.32       0.36       0.2        0.04      ]
 [0.16666667 0.83333333 0.         0.         0.        ]
 [0.0952381  0.66666667 0.19047619 0.         0.04761905]
 [0.1        0.3        0.3        0.3        0.        ]
 [0.07

In [ ]:
# b) Naive Bayes
result = ModelUtil.runNB()

In [ ]:
# c)Logistic Regression
result = ModelUtil.runLogReg()

In [ ]:
# d) SVM
result = ModelUtil.runSVM()

In [ ]:
# 3) Model Testing:

In [ ]:
# 4) Model Iterations and Improvements